# 06. Pytorch Transfer Learning.

transfer learning adalah teknik yang digunakan untuk mempercepat proses training model deep learning. Dengan menggunakan transfer learning, kita dapat menggunakan model yang sudah dilatih sebelumnya untuk menyelesaikan tugas yang berbeda. Dengan menggunakan model yang sudah dilatih sebelumnya, kita dapat menghemat waktu dan sumber daya yang dibutuhkan untuk melatih model dari awal.

- pretrained model = foundation model

In [4]:
import torch
import torchvision

print(torch.__version__)
print(torchvision.__version__)

2.5.1+cu124
0.20.1+cu124


In [5]:
## 1. Download going modular with pytorch 

import matplotlib.pyplot as plt
from going_modular import  data_setup, engine, model_builder, utils

In [6]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [7]:
!nvidia-smi

Tue Nov 26 22:17:37 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 552.22                 Driver Version: 552.22         CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce GTX 1050      WDDM  |   00000000:01:00.0 Off |                  N/A |
| N/A   44C    P0             N/A / ERR!  |       0MiB /   4096MiB |      1%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

## 1. Get data

kita perlu dataset pizza, steak, sushi untuk buat model transfer learning.

In [8]:
import requests
import zipfile
from pathlib import Path

# setup path to a data folder
data_path = Path("data/")
image_path = data_path / 'pizza_steak_sushi'

# if the image folder doesn't exist, download and prepare it...
if image_path.is_dir():
    print(f'{image_path} directory sudah ada, skipping download')
else:
    print(f'{image_path} tidak ada, mulai download')
    image_path.mkdir(parents=True, exist_ok=True)
    
# download the dataset from mrdbourke github
with open(data_path / 'pizza_steak_sushi.zip', mode="wb") as f:
    requests = requests.get("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip")
    print('Downloading dataset')
    f.write(requests.content)
    
# unzip dataset
with zipfile.ZipFile(data_path / 'pizza_steak_sushi.zip', 'r') as zip_ref:
    print('Unzipping dataset')
    zip_ref.extractall(image_path)

data\pizza_steak_sushi directory sudah ada, skipping download
Unzipping dataset


In [13]:
# setup directory path
train_dir = image_path / 'train'
test_dir = image_path / 'test'

train_dir, test_dir

(WindowsPath('data/pizza_steak_sushi/train'),
 WindowsPath('data/pizza_steak_sushi/test'))

## 2. Create datasets and dataloaders

udah ada data sekarang kita buat dataset dan dataloader

kita pake `data_setup.py` yang udah ada

In [14]:
from going_modular import data_setup

### 2.1 Create transform untuk `torchvision.models` (manual creation)


In [15]:
from torchvision import transforms

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

manual_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    normalize
])

In [16]:
train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(train_dir=train_dir, 
                                                                               test_dir=test_dir, 
                                                                               transform=manual_transform,
                                                                               batch_size=32)

train_dataloader, test_dataloader, class_names

(<torch.utils.data.dataloader.DataLoader at 0x284f67ec310>,
 ['pizza', 'steak', 'sushi'])

### 2.2 Create transform untuk `torchvision.models` (auto creation)

dari torchvision versi 0.13+ sudah ada fitur auto creation yang memungkinkan kita untuk membuat transform dengan mudah.

In [17]:
# get a set of pretrained model weigths
weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT # default artinya adalah best available weights
weights

EfficientNet_B0_Weights.IMAGENET1K_V1

In [18]:
# get the transforms used to create the model
auto_transform = weights.transforms()
auto_transform

ImageClassification(
    crop_size=[224]
    resize_size=[256]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BICUBIC
)

In [19]:
# create dataloader with the auto_transform
train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(train_dir=train_dir, 
                                                                               test_dir=test_dir, 
                                                                               transform=auto_transform,
                                                                               batch_size=32)

train_dataloader, test_dataloader, class_names

(<torch.utils.data.dataloader.DataLoader at 0x284f8a479a0>,
 ['pizza', 'steak', 'sushi'])